In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## API Overview

This notebook provides a demonstration of how to interact with graphrag as a library using the API as opposed to the CLI. Note that graphrag's CLI actually connects to the library through this API for all operations. 

In [2]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

In [3]:
PROJECT_DIRECTORY = Path.cwd().resolve().parent
PROJECT_DIRECTORY

PosixPath('/home/dongkyu/pdk_ws/study/graphrag/src')

## Prerequisite
As a prerequisite to all API operations, a `GraphRagConfig` object is required. It is the primary means to control the behavior of graphrag and can be instantiated from a `settings.yaml` configuration file.

Please refer to the [CLI docs](https://microsoft.github.io/graphrag/cli/#init) for more detailed information on how to generate the `settings.yaml` file.

### Generate a `GraphRagConfig` object

In [4]:
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

## Indexing API

*Indexing* is the process of ingesting raw text data and constructing a knowledge graph. GraphRAG currently supports plaintext (`.txt`) and `.csv` file formats.

## Build an index

In [5]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

# index_result is a list of workflows that make up the indexing pipeline that was run
for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

/home/dongkyu/.cache/pypoetry/virtualenvs/graphrag-VVzCahAR-py3.10/lib/python3.10/site-packages/graspologic/utils/utils.py:1142: UserWarning: Graph has at least one unweighted edge using weight_attribute "weight". Defaulting unweighted edges to "1.0"
  warnings.warn(
/home/dongkyu/.cache/pypoetry/virtualenvs/graphrag-VVzCahAR-py3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/dongkyu/.cache/pypoetry/virtualenvs/graphrag-VVzCahAR-py3.10/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
ERROR:graphrag.index.operations.summarize_communities.community_reports_extractor:error generating community report
Traceback (most recent call last):
  File "/home/dongkyu/.cache/pypoetry/virtualenvs/graphrag-VVzCahAR-py3.10/lib/python3.10/site-packages/fnll

Workflow Name: create_base_text_units	Status: success
Workflow Name: create_final_documents	Status: success
Workflow Name: extract_graph	Status: success
Workflow Name: finalize_graph	Status: success
Workflow Name: create_communities	Status: success
Workflow Name: create_final_text_units	Status: success
Workflow Name: create_community_reports	Status: error
[KeyError('community')]


## Query an index

To query an index, several index files must first be read into memory and passed to the query API. 

In [6]:
entities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)

response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="Who is Scrooge and what are his main relationships?",
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/dongkyu/pdk_ws/study/graphrag/src/output/entities.parquet'

The response object is the official reponse from graphrag while the context object holds various metadata regarding the querying process used to obtain the final response.

In [ ]:
print(response)

Digging into the context a bit more provides users with extremely granular information such as what sources of data (down to the level of text chunks) were ultimately retrieved and used as part of the context sent to the LLM model).

In [ ]:
pprint(context)  # noqa: T203